# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 416 (delta 164), reused 83 (delta 39), pack-reused 172 (from 1)
Receiving objects: 100% (416/416), 49.97 MiB | 42.46 MiB/s, done.
Resolving deltas: 100% (255/255), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_LOSO
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

2025-05-05 16:17:29.304963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746461849.632025      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746461849.714451      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from gmrrnet_adhd.models.GMRRNet import GMRRNet

model_name = 'GMRRNet'
model_args = {
    'num_kernels': 3,
    'nb_classes': 2,
    'Chans': 19,
    'Samples': 512,
    'norm_rate': 0.1,
    'alpha': 2
}


compile_args = {
    'loss':['binary_crossentropy', None], 
    'optimizer': 'adam',
    'loss_weights' : [0.9, 0.1],
    'metrics' : [['binary_accuracy'], [None]]
}

model = GMRRNet(**model_args)

model.summary()

I0000 00:00:1746461869.059922      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746461869.060808      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 19, 512)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 19, 512)        │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 512, 19)        │              0 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 512, 19)        │             38 │ reshape[0][0]          │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, 512, 19)        │          6,528 │ layer_normalization[0… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 512, 19)        │             38 │ transformer_encoder[0… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 19, 512, 1)     │              0 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_layer_1          │ (None, 19, 19, 1)      │              0 │ reshape_1[0][0]        │
│ (GaussianKernelLayer)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_layer_2          │ (None, 19, 19, 1)      │              0 │ reshape_1[0][0]        │
│ (GaussianKernelLayer)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_layer_3          │ (None, 19, 19, 1)      │              0 │ reshape_1[0][0]        │
│ (GaussianKernelLayer)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 19, 19, 5)      │             50 │ gaussian_layer_1[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 19, 19, 5)      │             50 │ gaussian_layer_2[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 19, 19, 5)      │             50 │ gaussian_layer_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 19, 19, 15)     │              0 │ conv2d[0][0],          │
│ (Concatenate)             │                        │                │ conv2d_1[0][0],        │
│                           │                        │                │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2D_2 (Conv2D)         │ (None, 19, 19, 3)      │            408 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 9,438 (36.87 KB)

 Trainable params: 9,426 (36.82 KB)

 Non-trainable params: 12 (48.00 B)

# Resultados - Leave One Subject Out

In [5]:
results = train_LOSO(GMRRNet, X, y, sbjs, model_args, compile_args, 1, 60, model_name)

Evaluando modelo para el sujeto #1: v107


I0000 00:00:1746461881.823873      71 service.cc:148] XLA service 0x7da04c00d5b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746461881.824848      71 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746461881.824869      71 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746461883.038608      71 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1746461891.886370      71 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step
Métricas para v107: {'accuracy': 1.0}

Evaluando modelo para el sujeto #2: v108
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
Métricas para v108: {'accuracy': 1.0}

Evaluando modelo para el sujeto #3: v109
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step
Métricas para v109: {'accuracy': 0.2786885245901639}

Evaluando modelo para el sujeto #4: v10p
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
Métricas para v10p: {'accuracy': 1.0}

Evaluando modelo para el sujeto #5: v110
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step
Métricas para v110: {'accuracy': 1.0}

Evaluando modelo para el sujeto #6: v111
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step
Métricas para v111: {'accuracy': 1.0}

Evaluando modelo para el sujeto #7: v112
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Métricas para v112: {'accuracy': 1.0}

Evaluando modelo para el sujeto #8: v113
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
Métricas para v113: {'accuracy': 1.0}

Evaluando modelo para el sujeto #9: v114
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step
Métricas para v114: {

In [6]:
# Extraer todos los valores de accuracy
accuracies = [v['accuracy'] for v in results.values()]

# Calcular el promedio
average_accuracy = sum(accuracies) / len(accuracies)

print(f"Accuracy promedio: {average_accuracy}")

Accuracy promedio: 0.9628194116059348


In [7]:
import pickle

with open(f'results_LOSO_{model_name}_1.pkl', 'wb') as f:
    pickle.dump(results, f)